<a href="https://colab.research.google.com/github/David3Emmanuel/Audio-Tokenizer-PoC/blob/main/Audio_Tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install torchcodec

In [2]:
from datasets import load_dataset

dataset = load_dataset("micsell/common_voice_en")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 2000
    })
})

In [5]:
%pip install snac

In [7]:
from snac import SNAC
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = SNAC.from_pretrained("hubertsiuzdak/snac_24khz").eval().to(device)
model

SNAC(
  (encoder): Encoder(
    (block): Sequential(
      (0): ParametrizedConv1d(
        1, 48, kernel_size=(7,), stride=(1,), padding=(3,)
        (parametrizations): ModuleDict(
          (weight): ParametrizationList(
            (0): _WeightNorm()
          )
        )
      )
      (1): EncoderBlock(
        (block): Sequential(
          (0): ResidualUnit(
            (block): Sequential(
              (0): Snake1d()
              (1): ParametrizedConv1d(
                48, 48, kernel_size=(7,), stride=(1,), padding=(3,), groups=48
                (parametrizations): ModuleDict(
                  (weight): ParametrizationList(
                    (0): _WeightNorm()
                  )
                )
              )
              (2): Snake1d()
              (3): ParametrizedConv1d(
                48, 48, kernel_size=(1,), stride=(1,)
                (parametrizations): ModuleDict(
                  (weight): ParametrizationList(
                    (0): _WeightNorm()
    

In [9]:
import torchcodec
test = dataset['test']
sample_audio = test[0]["audio"]

from IPython.display import Audio
sample_array = sample_audio["array"]
sample_rate = sample_audio["sampling_rate"]
Audio(data=sample_array, rate=sample_rate)

In [10]:
sample_array.shape

(87168,)

In [11]:
import torchaudio
import torch

sample_waveform = torch.tensor(sample_array).to(device).unsqueeze(0).unsqueeze(0)
if sample_rate != 24_000:
  sample_waveform = torchaudio.functional.resample(sample_waveform, sample_rate, 24000)

print(sample_waveform.shape)
Audio(data=sample_waveform[0,0].cpu(), rate=24_000)

torch.Size([1, 1, 130752])


In [12]:
with torch.inference_mode():
  codes = model.encode(sample_waveform)

print(f"Encoded into {len(codes)} streams.")
[tokens.shape for tokens in codes]

Encoded into 3 streams.


[torch.Size([1, 64]), torch.Size([1, 128]), torch.Size([1, 256])]

In [13]:
with torch.inference_mode():
  reconstructed = model.decode(codes)

reconstructed.shape

torch.Size([1, 1, 131072])

In [14]:
Audio(data=reconstructed[0, 0].cpu().numpy(), rate=24000)